This notebook converts **BPASS** `.dat` output files into FITS format compatible with **GalspecFitX**. The **BPASS** v2.2.1 models are available [here](https://bpass.auckland.ac.nz/9.html).

For guidance on converting **Starburst99** templates, see the `starburst99_conversion` notebook.

Instructions for incorporating new **Starburst99** and **BPASS** templates into the existing **GalspecFitX** libraries can be found in *Rivera et al. (2025)*.

## Imports

In [1]:
import os
import numpy as np
from astropy.io import fits

## Functions

In [2]:
def file_to_lam(filename, output_path, bpass_ver):
    """
    Converts a tab-delimited text file containing BPASS model output into a FITS file 
    containing the wavelength array. This function is designed for compatibility with 
    the GalSpecFitX library.

    Parameters:
    ----------
    filename : str
        Path to the input text file containing BPASS spectral data.
    output_path : str
        Directory where the output FITS file will be saved.
    bpass_ver : str
        Version identifier for the BPASS model (used in naming the output file).

    Returns:
    -------
    None

    Notes:
    -----
    - Only the first column of data is extracted and saved (typically wavelength).
    - Assumes each line of the input file is tab-separated.
    - The output FITS file is saved as: <output_path>/bpassv<bpass_ver>_lam.fits.
    """
    
    with open(filename) as file:
        lines = [line.split('\t')[0] for line in file]
        
    data = np.loadtxt(lines)
    
    new_hdul = fits.HDUList()
    new_hdul.append(fits.PrimaryHDU(data[:, 0]))
    
    bpass_lam_file = f"bpass_v{bpass_ver}_lam.fits"
    new_hdul.writeto(f"{output_path}/{bpass_lam_file}",overwrite=True)     
    
    return bpass_lam_file

In [3]:
def file_to_temp(lam, filename, output_path, star_form, star_evol, IMF_name, bpass_ver, Z, ages):
    """
    Converts tab-delimited spectral data from a BPASS model output file into individual FITS files 
    for different stellar population ages.

    Parameters:
    ----------
    lam : array_like
        Wavelength grid (used to segment the data by age).
    filename : str
        Path to the input file containing spectral data.
    output_path : str
        Root directory where the output FITS files will be saved.
    star_form : str
        Star formation history identifier (used in output file naming).
    star_evol : str
        Stellar evolution model identifier (used in output file naming).
    IMF_name : str
        Initial Mass Function identifier (used in output file naming).
    bpass_ver : str
        Version of the BPASS model used (included in output file naming).
    Z : str
        Metallicity value as a string (used in output file naming).
    ages : list of str
        List of age strings corresponding to different time steps in the model (used in output file naming).

    Notes:
    -----
    - Assumes the input file is tab-delimited and each column (after the first) represents data at a specific age.
    - Only the first tab-separated value from each line is used (likely path or relevant numerical value).
    - FITS files are saved using the format:
      'BPASS_{bpass_ver}_{star_form[:4]}_{star_evol[:3]}_{IMF_name}.Zp{Z}T{age}.fits'
    - Existing FITS files with the same name will be overwritten.
    """

    with open(filename) as file:
        lines = [line.split('\t')[0] for line in file]
        
    data = np.loadtxt(lines)
    
    for i in np.arange(0, len(ages)):
        new_hdul = fits.HDUList()
        new_hdul.append(fits.PrimaryHDU(data[:, i+1]))
        new_hdul.writeto(os.path.join(output_path, star_form, star_evol, IMF_name, f"BPASS_{bpass_ver}_{star_form[:4]}_{star_evol[:3]}_{IMF_name}.Zp{Z}T{ages[i]}.fits"),overwrite=True)

In [4]:
def get_filename_for_m(star_evol, IMF_name, m):
    """
    Generate the appropriate filename for a given metallicity value `m`, using the provided
    stellar evolution model and initial mass function (IMF) name.

    Parameters:
    - star_evol (str): The stellar evolution model name (e.g., "single", "binary").
    - IMF_name (str): The initial mass function name (e.g., "imf135all_100").
    - m (str or float): The metallicity value, which can be a float or string (e.g., "0.020", "0.0001", or "1e-4").

    Returns:
    - str: The corresponding filename, formatted as:
        - For m >= 0.001: 'spectra-{star_evol[:3]}-{IMF_name}.zXXX.dat'
          where XXX is metallicity × 1000, zero-padded to 3 digits.
        - For m < 0.001: 'spectra-{star_evol[:3]}-{IMF_name}.zemX.dat'
          where X is the absolute value of the exponent in scientific notation.
    """
    m_float = float(m)
    if m_float >= 0.001:
        m_int = int(round(m_float * 1000))
        return f"spectra-{star_evol[:3]}-{IMF_name}.z{m_int:03d}.dat"
    else:
        exp = int(f"{m_float:.0e}".split('e')[1])  # Extract exponent from scientific notation
        return f"spectra-{star_evol[:3]}-{IMF_name}.zem{-exp}.dat"   

### Set Parameters

The directory structure of the GalspecFitX library is broken down in Rivera et. a (2025). Please keep in mind that the following parameter strings should correspond to the name of an existing directory or will be used to identify the .dat files or for filenaming. 

In [5]:
# Main directory to contain BPASS templates
output_path = '/grp/hst/wfc3i/irivera/tsrc/GalSpecFitX/full_suite/BPASS/'

In [6]:
# Sub-directory names
star_form = 'instantaneous' # star formation (e.g. 'instantaneous' or 'continuous')
star_evol = 'binary' # star evolution (e.g. single or binary)
IMF_name = 'imf170_300' # Follows the naming convention of the .dat files

In [7]:
# BPASS version used to name file containing wavelength array (bpass_lam_file)
bpass_ver = '2.2.1'

# Metallicities - these labels are used to name the templates
metallicities = ["1e-4","1e-5", "0.001", "0.002", "0.003", "0.004", "0.006", "0.008", "0.010", "0.014", "0.020", "0.030", "0.040"]

## Format the ages 

The ages were created following the instructions in the BPASSv2.2.1_Manual.pdf, as quoted:
"Each file has 52 columns and 106 rows. The first column lists a wavelength in angstroms,
and each remaining column n (n>1) holds the model flux for the population at an age of
10^(6+0.1*(n-2)) years at that wavelength."

Ages are converted to Gyr.

In [8]:
# Path to .dat files for one star formation, star evolution, and IMF with the only difference between the .dat files being the metallicities.
dat_file_path = f'../../BPASS/bpass_templates/{star_evol}/{IMF_name}/'
one_dat_file = get_filename_for_m(star_evol, IMF_name, metallicities[0])
one_dat_file_path = f'{dat_file_path}/{one_dat_file}'

with open(one_dat_file_path) as file:
    lines = [line.split('\t')[0] for line in file]
    
    data = np.loadtxt(lines)

In [9]:
ages = []
    
for n in np.arange(2, data.shape[1]+1):
    ages.append((10**(6+0.1*(n-2))) / 1e9)

In [10]:
ages

[0.001,
 0.0012589254117941662,
 0.001584893192461114,
 0.001995262314968879,
 0.0025118864315095825,
 0.0031622776601683794,
 0.003981071705534969,
 0.005011872336272725,
 0.00630957344480193,
 0.007943282347242822,
 0.01,
 0.012589254117941663,
 0.01584893192461114,
 0.01995262314968879,
 0.025118864315095822,
 0.03162277660168379,
 0.03981071705534969,
 0.05011872336272725,
 0.0630957344480193,
 0.07943282347242822,
 0.1,
 0.12589254117941662,
 0.1584893192461111,
 0.19952623149688828,
 0.25118864315095824,
 0.31622776601683794,
 0.39810717055349687,
 0.5011872336272715,
 0.6309573444801944,
 0.7943282347242822,
 1.0,
 1.2589254117941662,
 1.584893192461111,
 1.9952623149688828,
 2.511886431509582,
 3.1622776601683795,
 3.981071705534969,
 5.011872336272715,
 6.309573444801943,
 7.943282347242822,
 10.0,
 12.589254117941714,
 15.848931924611108,
 19.952623149688826,
 25.11886431509582,
 31.622776601683793,
 39.810717055349855,
 50.11872336272715,
 63.09573444801943,
 79.432823472428

## Create the wavelength array and check the range for consistency

Use one .dat file to create the wavelength file. It shouldn't matter which since all the templates should have the same wavelength sampling.

In [11]:
bpass_lam_file = file_to_lam(one_dat_file_path, output_path, bpass_ver)

In [12]:
bpass_lam = fits.getdata(f"{output_path}/{bpass_lam_file}")

In [13]:
# Comparing this to what you expect was extracted from the .dat file is a good way to confirm correctness
bpass_lam

array([1.0000e+00, 2.0000e+00, 3.0000e+00, ..., 9.9998e+04, 9.9999e+04,
       1.0000e+05], dtype='>f8')

In [14]:
bpass_lam[0], bpass_lam[-1]

(1.0, 100000.0)

In [15]:
len(bpass_lam)

100000

## Create spectral templates from BPASS data files for use in GalSpecFitX

The loop below converts the fluxes column in the .dat files into FITS files for use in the code. This expects one star formation, star evolution, and IMF with the only difference between the .dat files being the metallicities.

In [16]:
for m in metallicities:
    file_name = get_filename_for_m(star_evol, IMF_name, m)
    file_path = f"{dat_file_path}/{file_name}"
    file_to_temp(bpass_lam, file_path, output_path, star_form, star_evol, IMF_name, bpass_ver, str(m), ages)

In [17]:
# Again another way to verify correctness is compare one template with its original .dat file
fits.getdata(f"{output_path}/{star_form}/{star_evol}/{IMF_name}/BPASS_{bpass_ver}_{star_form[:4]}_{star_evol[:3]}_{IMF_name}.Zp{metallicities[0]}T{ages[0]}.fits")

array([2.317866e-40, 2.317866e-40, 2.317866e-40, ..., 2.286993e-02,
       2.286902e-02, 2.286811e-02], dtype='>f8')